In [9]:
import asyncio, time, sys
import websockets, json
import numpy as np
import syncmetrics as syncm
from numpy_ringbuffer import RingBuffer

from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder
from pythonosc import udp_client
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [10]:
# OSC output config
local_ip = 'localhost'
local_port = 12000
output_address = '/diff'
client = udp_client.SimpleUDPClient(local_ip, local_port)

#    Send to Processing/MAX/Etc...
def msg(values):
    client.send_message(output_address, values[0])

#    Send to Processing/MAX/Etc...
def send_osc_bundle(values):
    bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)
    msg = osc_message_builder.OscMessageBuilder(address=output_address)
    for arg_to_add in values:
        msg.add_arg(arg_to_add)
    bundle.add_content(msg.build())
    bundle = bundle.build()
    client.send(bundle)

def output_labeled(json_data):
    print(json_data)
    for key, value in json_data:
        output_address = "/results/" + label
        msg = osc_message_builder.OscMessageBuilder(
            address=output_address)
        arg_to_add = value
        msg.add_arg(arg_to_add)
        msg.build()
        self.client.send(msg)
#     bundle = osc_bundle_builder.OscBundleBuilder(
#         osc_bundle_builder.IMMEDIATELY)
    

In [16]:
class Session:
    device_data = json.dumps({})
    uri = 'ws://localhost:9001'
    ws_timeout = 5.0
    num_users = 2

    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    window_A = np.zeros(win_size)
    window_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    
#     ema_buffers = RingBuffer(capacity=150, dtype=(np.float, num_users)) 
    Window_A = RingBuffer(capacity=win_size, dtype=(float))
    Window_B = RingBuffer(capacity=win_size, dtype=(float))

    def calculate_features(self):
        device_data = json.loads(self.device_data)
        device_data = [device_data["CH1"], device_data["CH2"]]
        self.Window_A.extendleft(device_data[0])
        self.Window_B.extendleft(device_data[1])
#         if self.Window_A.shape[0] % self.win_size == 0:
        if self.Window_A.shape[0] > self.win_size-1:
            buff_A = self.Window_A
            buff_B = self.Window_B
#             result_1=syncm.cos_similarity(buff_A, buff_B).flat[0]
            result_1=syncm.correlation_coeff(buff_A, buff_B).flat[0]
        
            #add results here
            results = [float(device_data[0][0]), float(device_data[1][0])]
            print(results)
            send_osc_bundle(results)

    async def ServerBIT_recieve(self, uri):
        async with websockets.connect(uri) as websocket:
            while True:
                self.device_data = await websocket.recv()
                self.calculate_features()
                await asyncio.sleep(0.0)

    def run(self):
        uri = self.uri
        print("Serving on {}".format(uri))
        try:
            asyncio.get_event_loop().run_until_complete(
                self.ServerBIT_recieve(uri))
        except Exception as e:
            print(e)
            pass
        finally:
            print("no connection, try again")
            exit

In [ ]:
session = Session()
session.run()

Serving on ws://localhost:9001
[30940.0, 26836.0]
[30784.0, 27143.0]
[30776.0, 27006.0]
[30896.0, 26964.0]
[30604.0, 26117.0]
[30500.0, 21407.0]
[30417.0, 20990.0]
[30692.0, 23663.0]
[34204.0, 27586.0]
[38900.0, 29474.0]
[40852.0, 30820.0]
[44356.0, 31630.0]
[51564.0, 29977.0]
[59368.0, 25359.0]
[62628.0, 24323.0]
[64196.0, 24104.0]
[64856.0, 22791.0]
[59984.0, 19733.0]
[52924.0, 20321.0]
[47380.0, 24599.0]
[42000.0, 26997.0]
[16918.0, 32598.0]
[4203.0, 37287.0]
[2022.0, 47846.0]
[8708.0, 41383.0]
[10693.0, 38237.0]
[12230.0, 36070.0]
[13596.0, 35781.0]
[16586.0, 38599.0]
[17712.0, 42095.0]
[18972.0, 43655.0]
[20500.0, 46170.0]
[21608.0, 47320.0]
[21316.0, 47024.0]
[19876.0, 47271.0]
[21791.0, 46692.0]
[26788.0, 46210.0]
[34009.0, 45722.0]
[40072.0, 44905.0]
[43968.0, 44036.0]
[48720.0, 42991.0]
[54156.0, 41815.0]
[57636.0, 40955.0]
[58920.0, 39744.0]
[61776.0, 38948.0]
[60292.0, 37755.0]
[57436.0, 37192.0]
[55302.0, 36834.0]
[54180.0, 35907.0]
[53180.0, 35495.0]
[51188.0, 34969.0]
[47

[29049.0, 24775.0]
[28700.0, 27271.0]
[28552.0, 29294.0]
[27630.0, 29791.0]
[27576.0, 31386.0]
[27892.0, 32065.0]
[27981.0, 31911.0]
[28064.0, 31615.0]
[28962.0, 31709.0]
[29933.0, 33031.0]
[30634.0, 37635.0]
[29420.0, 43458.0]
[27764.0, 35711.0]
[27321.0, 36893.0]
[27629.0, 38718.0]
[44940.0, 36566.0]
[40444.0, 33572.0]
[25590.0, 33082.0]
[12258.0, 33358.0]
[5450.0, 33743.0]
[2467.0, 33164.0]
[2356.0, 32839.0]
[12360.0, 34423.0]
[33373.0, 33486.0]
[49670.0, 32766.0]
[57716.0, 32146.0]
[57804.0, 31424.0]
[57188.0, 31284.0]
[50192.0, 30982.0]
[38994.0, 30973.0]
[18520.0, 30492.0]
[9344.0, 30160.0]
[33200.0, 29740.0]
[50466.0, 29729.0]
[47862.0, 30122.0]
[48948.0, 31239.0]
[39808.0, 26815.0]
[24206.0, 25903.0]
[5552.0, 25124.0]
[10408.0, 24212.0]
[16040.0, 24035.0]
[23580.0, 25646.0]
[30932.0, 25283.0]
[40060.0, 25650.0]
[46368.0, 23897.0]
[43176.0, 25255.0]
[36584.0, 25116.0]
[32861.0, 24428.0]
[29941.0, 24687.0]
[51796.0, 38860.0]
[48688.0, 42179.0]
[37240.0, 44095.0]
[18736.0, 45293.0

[19564.0, 34055.0]
[18764.0, 39272.0]
[18980.0, 38540.0]
[20125.0, 37438.0]
[21376.0, 36231.0]
[20538.0, 35252.0]
[19950.0, 34220.0]
[19920.0, 32356.0]
[20188.0, 27840.0]
[20486.0, 26791.0]
[20176.0, 30201.0]
[19928.0, 27748.0]
[19932.0, 26342.0]
[20050.0, 26168.0]
[19928.0, 25403.0]
[20064.0, 24042.0]
[20180.0, 24583.0]
[20524.0, 27004.0]
[21108.0, 28135.0]
[22504.0, 27631.0]
[26064.0, 26087.0]
[35738.0, 25273.0]
[45820.0, 25130.0]
[51908.0, 24871.0]
[52576.0, 26233.0]
[44584.0, 29370.0]
[39144.0, 31506.0]
[34340.0, 28020.0]
[37476.0, 27879.0]
[38560.0, 29287.0]
[37916.0, 30270.0]
[34024.0, 31108.0]
[32576.0, 31486.0]
[31888.0, 33319.0]
[30808.0, 36159.0]
[29816.0, 35963.0]
[28296.0, 39047.0]
[27286.0, 43868.0]
[27292.0, 45551.0]
[28616.0, 51839.0]
[28317.0, 52727.0]
[27036.0, 44239.0]
[26080.0, 32027.0]
[25452.0, 25986.0]
[25170.0, 26878.0]
[23276.0, 26235.0]
[18728.0, 24898.0]
[18348.0, 23974.0]
[18632.0, 24799.0]
[19613.0, 26302.0]
[19676.0, 27599.0]
[19184.0, 27940.0]
[19212.0, 27

In [ ]:
class Session:
    device_data = json.dumps({})
    uri = 'ws://localhost:9001'
    ws_timeout = 5.0
    num_users = 2

    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    window_A = np.zeros(win_size)
    window_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    
    ema_buffers = RingBuffer(capacity=150, dtype=(float, num_users)) 
    feature_buffers = RingBuffer(capacity=win_size, dtype=(float, num_users))

    def calculate_features(self):
        device_data = json.loads(self.device_data)
        device_data = [np.mean(device_data["CH1"]), np.mean(device_data["CH2"])]
        self.ema_buffers.appendleft(device_data)
        sigA = (self.ema_buffers[0][0] - self.ema_buffers[0][-1])
        sigB = (self.ema_buffers[1][0] - self.ema_buffers[1][-1])
        self.feature_buffers.appendleft([sigA, sigB])
#         self.feature_buffers.appendleft([float(device_data["CH1"][0]), float(device_data["CH2"][0])])
#         if self.feature_buffers.shape[0] > self.win_size-1:
#             buff_A = np.array([i[0] for i in np.array(self.feature_buffers)])
#             buff_B = np.array([i[1] for i in np.array(self.feature_buffers)])
        if True:
            result_1=(abs(sigA-sigB)/500.0)
#             result_2=syncm.cos_similarity(buff_A, buff_B).flat[0]
#             result_3=syncm.correlation_coeff(buff_A, buff_B).flat[0]
        
            #add results here
            results = [result_1]
            print(results)
            msg(results)

    async def ServerBIT_recieve(self, uri):
        async with websockets.connect(uri) as websocket:
            while True:
                self.device_data = await websocket.recv()
                self.calculate_features()
                await asyncio.sleep(0.0)

    def run(self):
        uri = self.uri
        print("Serving on {}".format(uri))
        try:
            asyncio.get_event_loop().run_until_complete(
                self.ServerBIT_recieve(uri))
        except Exception as e:
            print(e)
            pass
        finally:
            print("no connection, try again")
            exit